In [1]:
#Step 1 : Data Collection
#Step 2 : Feature Extraction | Faces in this case
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import os

In [2]:
import os
def capture_and_crop_eyes(video_path):
    # Load the Haar cascade for eye detection
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if the video was opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Create a folder to store the cropped eye images
    if not os.path.exists('cropped'):
        os.makedirs('cropped')
    
    frame_count = 0
    eye_count = 0

    while True:
        # Read a frame
        ret, frame = cap.read()
        
        # If no frame is returned, the video has ended
        if not ret:
            break
        
        frame_count += 1

        # Convert the frame to grayscale for eye detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect eyes in the frame
        eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Crop and save each detected eye
        for (x, y, w, h) in eyes:
            eye = frame[y:y+h, x:x+w]
            eye_count += 1
            eye_filename = f'cropped/eye_{eye_count}.png'
            cv2.imwrite(eye_filename, eye)

    # Release the video capture object
    cap.release()

    print(f"Processed {frame_count} frames and saved {eye_count} cropped eye images.")

# Example usage:
video_path = r'video\WhatsApp Video 2024-07-31 at 23.12.48_43b230b9.mp4'
capture_and_crop_eyes(video_path)

Processed 193 frames and saved 580 cropped eye images.


In [3]:
def create_dataset_mapping(dataset_path):
    direction_folders = ['down', 'left', 'right', 'up']
    direction_labels = {direction: idx for idx, direction in enumerate(direction_folders)}
    
    print("Direction labels:")
    for direction, label in direction_labels.items():
        print(f"{direction}: {label}")
    
    # Create a list to store the image paths and their corresponding labels
    dataset = []

    for direction in direction_folders:
        folder_path = os.path.join(dataset_path, direction)
        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(".png"):  # or any other image format
                    file_path = os.path.join(folder_path, filename)
                    dataset.append((file_path, direction_labels[direction]))

    return dataset, direction_labels

# Example usage:
dataset_path = 'dataset3'
dataset, direction_labels = create_dataset_mapping(dataset_path)

# Print the dataset
print("\nDataset:")
for file_path, label in dataset:
    print(f"{file_path}: {label}")


Direction labels:
down: 0
left: 1
right: 2
up: 3

Dataset:
dataset3\down\eye_465.png: 0
dataset3\down\eye_466.png: 0
dataset3\down\eye_468.png: 0
dataset3\down\eye_472.png: 0
dataset3\down\eye_473.png: 0
dataset3\down\eye_474.png: 0
dataset3\down\eye_477.png: 0
dataset3\down\eye_479.png: 0
dataset3\down\eye_480.png: 0
dataset3\down\eye_481.png: 0
dataset3\down\eye_482.png: 0
dataset3\down\eye_483.png: 0
dataset3\down\eye_484.png: 0
dataset3\down\eye_485.png: 0
dataset3\down\eye_486.png: 0
dataset3\down\eye_487.png: 0
dataset3\down\eye_488.png: 0
dataset3\down\eye_489.png: 0
dataset3\down\eye_490.png: 0
dataset3\down\eye_491.png: 0
dataset3\down\eye_492.png: 0
dataset3\down\eye_493.png: 0
dataset3\down\eye_494.png: 0
dataset3\down\eye_495.png: 0
dataset3\down\eye_496.png: 0
dataset3\down\eye_497.png: 0
dataset3\down\eye_498.png: 0
dataset3\down\eye_499.png: 0
dataset3\down\eye_500.png: 0
dataset3\down\eye_501.png: 0
dataset3\down\eye_502.png: 0
dataset3\down\eye_503.png: 0
dataset3\down

In [4]:
def w2d(img):
    imArray = img
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    
    imArray =  np.float32(imArray)   
    imArray /= 255
  

    return imArray

In [5]:
import random
from sklearn.model_selection import train_test_split

# Shuffle dataset
random.shuffle(dataset)

# Split dataset into training and validation sets
train_ratio = 0.8  # 80% training, 20% validation
train_size = int(len(dataset) * train_ratio)

train_data = dataset[:train_size]
val_data = dataset[train_size:]

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")


Training set size: 456
Validation set size: 115


In [6]:
import cv2
import numpy as np

# Example function to extract features (e.g., resizing and flattening)
def extract_features(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (32, 32))  # Resize image to (32, 32)
    features = img.flatten()  # Flatten image into a 1D array
    return features

# Example usage to extract features from dataset
X_train = []
y_train = []

for file_path, label in train_data:
    features = extract_features(file_path)
    X_train.append(features)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Similarly, extract features for validation data (val_data)
X_val = []
y_val = []

for file_path, label in val_data:
    features = extract_features(file_path)
    X_val.append(features)
    y_val.append(label)

X_val = np.array(X_val)
y_val = np.array(y_val)

# Print shapes to verify
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")


X_train shape: (456, 1024), y_train shape: (456,)
X_val shape: (115, 1024), y_val shape: (115,)


In [7]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Example SVM training
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)
svm_pred = svm_clf.predict(X_val)
svm_acc = accuracy_score(y_val, svm_pred)
print(f"SVM Accuracy: {svm_acc}")

# Example Logistic Regression training
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_val)
lr_acc = accuracy_score(y_val, lr_pred)
print(f"Logistic Regression Accuracy: {lr_acc}")

# Example Random Forest training
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_val)
rf_acc = accuracy_score(y_val, rf_pred)
print(f"Random Forest Accuracy: {rf_acc}")


SVM Accuracy: 0.9565217391304348
Logistic Regression Accuracy: 0.9304347826086956
Random Forest Accuracy: 0.9391304347826087


In [8]:
from sklearn.svm import SVC  # for classification tasks
from sklearn.svm import SVR  # for regression tasks
from joblib import dump, load

# Example: Train a Random Forest model
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
# Assuming X_train and y_train are your training data
rf_clf.fit(X_train, y_train)

# Define the path where you want to save the model
model_path = 'SVMmodel/SVM_model.joblib'

# Save the model to the specified path
dump(svm_clf, model_path)

['SVMmodel/SVM_model.joblib']

In [9]:
# Load the model from the saved file
loaded_model = load(model_path)

# Now you can use loaded_model for predictions or further operations


In [10]:
import pyautogui

pyautogui.FAILSAFE = True  # Enable fail-safe (default behavior)
pyautogui.FAILSAFE_INTERVAL = 2.0  # Adjust the fail-safe check interval if needed
pyautogui.PAUSE = 0.1  # Adjust the pause duration between PyAutoGUI actions if needed
pyautogui.FAILSAFE_POINTS = [(0, 0)]  # Adjust the fail-safe corner points to ignore

# Adjust the distance before the fail-safe triggers
pyautogui.FAILSAFE_MARGIN = 10

In [11]:
import cv2
import pyautogui
from joblib import load  # Import load function from joblib

pyautogui.FAILSAFE = False  # Disable PyAutoGUI fail-safe

def extract_eye_region(frame):
    # Implement eye region extraction here
    # Example: Using Haar cascades for eye detection
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)
    
    if len(eyes) > 0:
        (ex, ey, ew, eh) = eyes[0]
        eye_region = frame[ey:ey+eh, ex:ex+ew]
        return eye_region
    else:
        return None

def predict_direction(frame, model):
    # Extract features from the eye region (assuming your feature extraction method)
    features = extract_features(frame)
    
    if features is not None:
        # Predict direction using the trained model
        prediction = model.predict([features])[0]
        
        # Map integer prediction to direction label
        direction_labels = {0: 'down',1: 'left', 2: 'right', 3: 'up'}
        predicted_direction = direction_labels.get(prediction, None)
        
        return predicted_direction
    else:
        return None

def extract_features(frame):
    # Implement feature extraction here
    # Example: Convert to grayscale and resize to (32, 32)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (32, 32))
    flattened = resized.flatten()
    return flattened

def move_cursor_with_eye_direction(video_path, model):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    screen_width, screen_height = pyautogui.size()
    cursor_step = 50  # Number of pixels to move the cursor
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame to extract eye region
        eye_region = extract_eye_region(frame)
        
        if eye_region is not None:
            # Predict direction
            direction = predict_direction(eye_region, model)
            
            # Assuming 'direction' is the variable holding the predicted direction
            if direction is not None:
                # Move cursor based on predicted direction
                if direction == 'up':
                    pyautogui.moveRel(0, -cursor_step)  # Move cursor up by cursor_step pixels
                elif direction == 'down':
                    pyautogui.moveRel(0, cursor_step)  # Move cursor down by cursor_step pixels
                elif direction == 'left':
                    pyautogui.moveRel(-cursor_step, 0)  # Move cursor left by cursor_step pixels
                elif direction == 'right':
                    pyautogui.moveRel(cursor_step, 0)  # Move cursor right by cursor_step pixels
                
                # Display direction text on the frame
                cv2.putText(frame, f"Direction: {direction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # Display the frame
        cv2.imshow('Frame', frame)
        
        # Press 'q' on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    
    # Release video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Example usage with your video file
video_path = r'video\WhatsApp Video 2024-07-31 at 23.12.48_43b230b9.mp4'

# Load your trained SVM model
model_path = r'SVMmodel\SVM_model.joblib'
svm_clf = load(model_path)  # Load your trained model file using joblib

# Call the function to move cursor based on eye direction in real-time
move_cursor_with_eye_direction(video_path, svm_clf)
